# Part 1: Data Preparation

In [1]:
import pandas as pd
import spacy
import matplotlib.pyplot as plt

In [2]:
# english model
nlp = spacy.load("en_core_web_sm")

### Data cleaning

In [3]:
from data_process import extract_yoda_lines, clean_text
import re

In [4]:
yoda_raw = extract_yoda_lines('data/yoda-corpus.csv')
yoda_raw_df = pd.DataFrame(yoda_raw)
print(f"Raw rows: {len(yoda_raw_df):,}")

Raw rows: 100


In [5]:
yoda_df = yoda_raw_df.dropna(subset=["text"]).drop_duplicates(subset=["text"]).reset_index(drop=True)
print(f"After dropping nulls & duplicates: {len(yoda_df):,}")

After dropping nulls & duplicates: 99


In [6]:
# data cleaning

yoda_df["clean"] = yoda_df["text"].apply(clean_text)

yoda_df["token_count"] = yoda_df["clean"].str.split().apply(len)
yoda_df = yoda_df[yoda_df["token_count"] >= 2].reset_index(drop=True)
print(f"After cleaning & filtering short lines: {len(yoda_df):,}")


After cleaning & filtering short lines: 98


In [7]:
# Quick sanity check: sample few cleaned lines
display(yoda_df.sample(5)[["text","clean"]])

,text,clean
39,A prophecy . . . that misread could have been.,a prophecy . . . that misread could have been.
28,Premonitions . . . premonitions . . . Hmmmm ....,premonitions . . . premonitions . . . hmmmm . ...
87,"How to join the Force, he will train you. You...","how to join the force, he will train you. your..."
33,Train yourself to let go of everything you fe...,train yourself to let go of everything you fea...
70,"Faith in your new apprentice, misplaced may b...","faith in your new apprentice, misplaced may be..."


In [8]:
# Save cleaned version for reuse
# yoda_df.to_csv("data/yoda_clean.csv", index=False)

## Generate Normal English for Yoda speech

In [9]:
from chat_gpt_call import generate_normal_english_and_mapping
import asyncio

In [10]:
progress_counter = 0
progress_lock = asyncio.Lock()

async def generate_with_progress(yoda_sentence: str, idx: int, total: int):
    """
    Wrapper around generate_normal_english_and_mapping that prints progress.
    """
    global progress_counter
    try:
        result = await generate_normal_english_and_mapping(yoda_sentence)
    except Exception as e:
        result = e
    async with progress_lock:
        progress_counter += 1
        print(f"Processed {progress_counter}/{total} sentences (Task {idx})", end="\r")
    return result

In [11]:
async def process_all_yoda_sentences(yoda_df: pd.DataFrame) -> pd.DataFrame:
    tasks = []
    total = len(yoda_df)
    
    # Create a list of tasks with progress wrapper
    for idx, row in yoda_df.iterrows():
        yoda_sentence = row["text"]
        tasks.append(generate_with_progress(yoda_sentence, idx, total))
    
    # Run all tasks concurrently; capture exceptions by setting return_exceptions=True
    responses = await asyncio.gather(*tasks, return_exceptions=True)
    
    results = []
    for idx, res in enumerate(responses):
        yoda_sentence = yoda_df.iloc[idx]["text"]
        if isinstance(res, Exception):
            print(f"\nError processing row {idx}: {res}")
            results.append({
                "original": yoda_sentence,
                "normal_english": None,
                "mapping": None
            })
        else:
            results.append({
                "original": yoda_sentence,
                "normal_english": res.normal_english,
                "mapping": res.mapping
            })
    
    return pd.DataFrame(results)

In [15]:
# loop = asyncio.get_event_loop()
# normalized_df = loop.run_until_complete(process_all_yoda_sentences(yoda_df))

final_df1 = await process_all_yoda_sentences(yoda_df[:20])

In [14]:
final_df2 = await process_all_yoda_sentences(yoda_df[20:40])

In [16]:
final_df3 = await process_all_yoda_sentences(yoda_df[40:60])


In [17]:
final_df4 = await process_all_yoda_sentences(yoda_df[60:80])


In [18]:
final_df5 = await process_all_yoda_sentences(yoda_df[80:])

In [19]:
final_df = pd.concat([final_df1, final_df2, final_df3, final_df4, final_df5], ignore_index=True)

In [20]:
print(f"Final rows: {len(final_df):,}")

display(final_df.sample(5))

Final rows: 98


,original,normal_english,mapping
37,"Go, I will. Good relations with the Wookiees,...",I will go. I have good relations with the Wook...,"[1, 2, 0, 8, 9, 3, 4, 5, 6, 7]"
71,"Destroy you I will, just as Master Kenobi, yo...","I will destroy you, just as your apprentice wi...","[2, 3, 0, 1, 4, 5, 8, 9, 10, 11, 6, 7]"
62,"To fight this Lord Sidious, strong enough, yo...",You are not strong enough to fight this Lord S...,"[7, 8, 9, 5, 6, 0, 1, 2, 3, 4]"
96,"Remember, a Jedi's strength flows from the For...","Remember, a Jedi's strength flows from the For...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 9]"
11,"See through you, we can.",We can see through you.,"[3, 4, 0, 1, 2]"


In [41]:
yoda_df.iloc[12]["text"]

# find the max length of text and which row it is
max_index = yoda_df["text"].str.len().idxmax()
max_row = yoda_df.iloc[max_index]
print(f"Row with max length: {max_row['text']}")
print(f"Max length of text: {max_row['text'].str.len()}")
max_len = yoda_df["text"].str.len().max()
print(f"Max length of text: {max_len}")

Row with max length:  Death is a natural part of life. Rejoice for those around you who transform into the Force. Mourn them, do not. Miss them, do not. Attachment leads to jealousy. The shadow of greed, that is.


AttributeError: 'str' object has no attribute 'str'

# Save the Yoda speech to a file

In [23]:
import json

In [25]:
df_to_save = final_df.copy()
df_to_save['mapping'] = df_to_save['mapping'].apply(json.dumps)

df_to_save.to_csv("data/yoda_normalized.csv", index=False)

In [26]:
# to load:

In [ ]:
import pandas as pd
import json

df_loaded = pd.read_csv("data/yoda_normalized.csv")

df_loaded['mapping'] = df_loaded['mapping'].apply(json.loads)